In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import glob
import json

In [ ]:
duration = [4.5, 4, 1, 8, 4, 4.5, 6]
with plt.xkcd():
    plt.boxplot(duration)
    plt.title("Time taken to do this exercise")
    plt.ylabel("hours")
    plt.xticks([1,], ["you", ])
    plt.show()

In [ ]:
dir_ = "./PERFORMANCEMATRIX/"

In [ ]:
id2 = 269 # 254
id1 = 336

loss_dc1 = {}
for fl in glob.glob("%s/%s_*" % (dir_, id1)):
    with open(fl, "r") as fh:
        line = fh.readlines()[0]
        line = json.loads(line)
        loss_dc1[line["run_on"]] = line["loss"]

loss_dc2 = {}
for fl in glob.glob("%s/%s_*" % (dir_, id2)):
    with open(fl, "r") as fh:
        line = fh.readlines()[0]
        line = json.loads(line)
        loss_dc2[line["run_on"]] = line["loss"]
        
data_ids = list(loss_dc1.keys())
data_ids = np.sort(data_ids)

y1 = [loss_dc1[i] for i in data_ids]
y2 = [loss_dc2[i] for i in data_ids]
y1 = np.array(y1)
y2 = np.array(y2)

print("A: Mean, std: %g +- %g" % (np.mean(y1), np.std(y1)))
print("A: Quartiles: %f, %f, %f" % (np.percentile(y1, 25),
                                    np.percentile(y1, 50),
                                    np.percentile(y1, 75)))
print("B: Mean, std: %g +- %g" % (np.mean(y2), np.std(y2)))
print("B: Quartiles: %f, %f, %f" % (np.percentile(y2, 25),
                                    np.percentile(y2, 50), 
                                    np.percentile(y2, 75)))
#np.savetxt("data.csv", np.vstack([y1, y2]).T, delimiter=",")

In [ ]:
# Scatter plot
thresh = 0.1
idx1 = (y1 > y2+thresh)
plt.scatter(y1[idx1], y2[idx1], label="B is better")
print("A is better than B on %d datasets" % np.sum(idx1))


idx2 = (y2 > y1+thresh)
plt.scatter(y1[idx2], y2[idx2], label="A is better")
print("B is better than A on %d datasets" % np.sum(idx2))

plt.scatter(y1, y2, marker="+", zorder=0, label="equal")
print("A and B perform comparable on %d datasets" % (y1.shape[0] - np.sum(idx1) - np.sum(idx2)))
plt.plot([0, 1], [0,1], c='k', zorder=0)
plt.plot([0, 1-thresh], [thresh, 1], c='k', linestyle=":", zorder=0)
plt.plot([thresh, 1], [0, 1-thresh], c='k', linestyle=":", zorder=0)
plt.xlabel("Error of A")
plt.ylabel("Error of B")
plt.legend()

In [ ]:
# eCDF
plt.step(np.sort(y1[::10]), np.arange(1, len(y1[::10])+1) / (len(y1[::10])+1), label="A")
plt.step(np.sort(y2[::10]), np.arange(1, len(y2[::10])+1) / (len(y2[::10])+1), label="B")

plt.step(np.sort(y1[::10]), np.arange(len(y1[::10])) / (len(y1[::10])), label="A")
plt.step(np.sort(y2[::10]), np.arange(len(y2[::10])) / (len(y2[::10])), label="B")

plt.legend()
plt.xlabel("Error")
plt.ylabel("P(X<L)")
plt.plot([0.4, 0.4], [1, 0], c='k', zorder=0, linestyle=":")
plt.show()
frac = np.sum(y1 <= 0.4)/len(y1)
print("Probability of A to achieve an error lower than 0.4: %f" % frac)
frac = np.sum(y2 <= 0.4)/len(y2)
print("Probability of B to achieve an error lower than 0.4: %f" % frac)

In [ ]:
# Boxplot and violin plot
# See also https://matplotlib.org/gallery/statistics/boxplot_vs_violin.html
plt.boxplot([y1, y2], labels=["A", "B"])
plt.ylabel("Error")
plt.xlabel("Dataset")
plt.show()

plt.violinplot([y1, y2])
plt.xticks([1, 2], ["A", "B"])
plt.xlabel("Dataset")
plt.ylabel("Error")
plt.show()

In [ ]:
# Paired Permutation test
print("H_0: A and B have equal means")
print("H_1: The mean of A is better (smaller) than that of B")

t = np.mean(y1) - np.mean(y2)
print("Test statistic t: %f" % t)
rng = np.random.RandomState(1)
s = []
for i in range(10000):
    ind_dataset1 = []
    ind_dataset2 = []
    for i,j in zip(y1, y2):
        if rng.random_sample() > 0.5:
            ind_dataset1.append(i)
            ind_dataset2.append(j)
        else:
            ind_dataset1.append(j)
            ind_dataset2.append(i)
    m1 = np.mean(ind_dataset1)
    m2 = np.mean(ind_dataset2)
    s.append(m1-m2)
s = np.array(s)

print("Samples with s<t: %d (of %d)" % (np.sum(s<t), len(s)))
p = np.sum(s<t)/len(s)
print("Then, p-value %f < alpha?" % p)

In [ ]:
# Paired Permutation test, other alternative hypothesis

print("H_0: A and B have equal means")
print("H_1: The mean of B is better (smaller) than that of A")

t = np.mean(y2) - np.mean(y1)
print("Test statistic t: %f" % t)
rng = np.random.RandomState(1)
s = []
for i in range(10000):
    ind_dataset1 = []
    ind_dataset2 = []
    for i,j in zip(y2, y1):
        if rng.random_sample() > 0.5:
            ind_dataset1.append(i)
            ind_dataset2.append(j)
        else:
            ind_dataset1.append(j)
            ind_dataset2.append(i)
    m1 = np.mean(ind_dataset1)
    m2 = np.mean(ind_dataset2)
    s.append(m2-m1)
s = np.array(s)

plt.hist(s)
plt.title("Distribution of test statistics")
plt.ylabel("number of permutations")
plt.xlabel("m1 - m2")
plt.show()

print("Samples with s<t: %d (of %d)" % (np.sum(s<t), len(s)))
p = np.sum(s<t)/len(s)
print("Then, p-value %f < alpha?" % p)

In [ ]:
# Paired Permutation test, other alternative hypothesis
print("H_0: A and B have equal means")
print("H_1: The mean of B is better/worse (smaller/higher) than that of A")

t = np.abs(np.mean(y2) - np.mean(y1))
print("Test statistic t: %f" % t)
rng = np.random.RandomState(1)
s = []
for i in range(10000):
    ind_dataset1 = []
    ind_dataset2 = []
    for i,j in zip(y2, y1):
        if rng.random_sample() > 0.5:
            ind_dataset1.append(i)
            ind_dataset2.append(j)
        else:
            ind_dataset1.append(j)
            ind_dataset2.append(i)
    m1 = np.mean(ind_dataset1)
    m2 = np.mean(ind_dataset2)
    s.append(np.abs(m2-m1))
s = np.array(s)

plt.hist(s)
plt.title("Distribution of test statistics")
plt.ylabel("number of permutations")
plt.xlabel("m1 - m2")
plt.show()

print("Samples with s<t: %d (of %d)" % (np.sum(s<t), len(s)))
p = np.sum(s<t)/len(s)
print("Then, p-value %f < alpha?" % p)

In [ ]:
# Permutation test (non-paired)
t = np.mean(y1) - np.mean(y2)
print("Test statistic t: %f" % t)
rng = np.random.RandomState(1)
s = []
for i in range(10000):
    ind_dataset = np.hstack([y1, y2])
    rng.shuffle(ind_dataset)
    m1 = np.mean(ind_dataset[:len(y1)])
    m2 = np.mean(ind_dataset[-len(y2):])
    s.append(m1-m2)
plt.hist(s)
plt.title("Distribution of test statistics")
plt.ylabel("number of permutations")
plt.xlabel("m1 - m2")
plt.show()

print("Samples with s<t: %d (of %d)" % (np.sum(s<t), len(s)))
p = np.sum(s<t)/len(s)
print("Then, p-value %f < alpha?" % p)

In [ ]:
# Alternative implementation, not needed

# data shape: string index, float performance data
n_points = len(y1)

# convert data to numpy array
data = np.vstack([y1, y2]).T
#print(data.shape)
# data[:, 1] += np.random.normal(loc=0, scale=1, size=(n_points,))

# compute original test statistics
test_stat = np.mean(y1) - np.mean(y2)
#print(test_stat)
# permutate columns in the data and compute test statistics for each
# permutation
runs = 1000
stats = []
for r in range(runs):
   perm = np.apply_along_axis(np.random.permutation, 1, data)
   test_stat_perm = np.mean(perm[:, 0]) - np.mean(perm[:, 1])
   stats.append(test_stat_perm)

# p value is now how many of those test statistcs are smaller than the
# original (not permutated) test statistics
# for equal means, this should be about 0.5
s_set = np.array(stats)
p_value = np.sum(s_set < test_stat) / runs
print(p_value)